In [1]:
!pip install sklearn
!pip install lightgbm


  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Ivan\AppData\Local\Temp\pip-install-b7qm7rgc\sklearn_5b0e257b46a64dd1bfd97ff8aa6cf0b8\setup.py", line 10, in <module>
          LONG_DESCRIPTION = f.read()
        File "D:\Programms\Python3.10\lib\encodings\cp1252.py", line 23, in decode
          return codecs.charmap_decode(input,self.errors,decoding_table)[0]
      UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 7: character maps to <undefined>
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the packa


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from scipy.sparse import hstack

features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])


train_features.fillna(0, inplace=True)
test_features.fillna(0, inplace=True)
validation_features.fillna(0, inplace=True)


print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [3]:
train_features.columns
validation_features.columns

Index(['filename', 'cell_type', 'cell_number', 'execution_count',
       'linesofcomment', 'linesofcode', 'variable_count', 'function_count',
       'text/plain', 'image/png', 'text/html', 'execute_result',
       'display_data', 'stream', 'error', 'text', 'comment',
       'code_line_before', 'code_line_after', 'markdown_heading',
       'packages_info', 'primary_label', 'helper_functions', 'load_data',
       'data_exploration', 'data_preprocessing', 'evaluation', 'modelling',
       'prediction', 'result_visualization', 'save_results', 'comment_only'],
      dtype='object')

In [4]:

target_drop = ["primary_label", "load_data", "helper_functions", 
          "data_preprocessing", "data_exploration", "modelling", 
          "prediction", "evaluation", "result_visualization", 
          "save_results", "comment_only"]

target = train_features["primary_label"]
train_features.drop(columns=target, inplace=True)

In [5]:
validation_features["code_line_before"].sample(10)

1716    [finaldf['start_day'] = pd.DatetimeIndex(final...
1818                                             [%reset]
343               [col_diff = df_tmp[df_tmp.d > 350].col]
1231                         [sns.distplot(ROTMM['HDD'])]
224                                    [print(fd_stores)]
875                                      [mapping.head()]
1795    [pyplot.imshow(helper.images_square_grid(mnist...
261                            [gs.fit(X_train, y_train)]
1104               [data = parse_reddit(reddit, the_url)]
1174                           [H_colors = add_colors(H)]
Name: code_line_before, dtype: object

In [6]:
train_features.text[1]

train_features["text"] = train_features["text"].apply(lambda x: " ".join(x))
validation_features["text"] = validation_features["text"].apply(lambda x: " ".join(x))
test_features["text"] = test_features["text"].apply(lambda x: " ".join(x))


In [7]:
validation_features["text"] 

0       import matplotlib.pyplot as plt import numpy a...
1           length = 80  # x range depth = 200  # z range
2       model = 1 + np.tri(depth, length, -depth//3) p...
3       model[:depth//3,:] = 0 plt.imshow(model) plt.c...
4       rocks = np.array([[2700, 2750], [2400, 2450], ...
                              ...                        
1922                   print(num_mistakes_dict['177'][1])
1923    print(num_mistakes_dict['279'][1]) print("----...
1924    print(num_mistakes_dict['2854'][1]) print("---...
1925    error_type_dict = {} for key in error_dict: fo...
1926    misspelled_words = {} for key in error_dict: f...
Name: text, Length: 1927, dtype: object

In [8]:
train_columns = ['cell_number', 'execution_count', 'linesofcomment', 'linesofcode', 
                 'variable_count', 'function_count', 'display_data', 'stream', 'error']

In [9]:
target.value_counts(normalize=True)

primary_label
data_exploration        0.285273
data_preprocessing      0.239328
modelling               0.158066
helper_functions        0.080062
load_data               0.074404
result_visualization    0.050060
evaluation              0.039945
prediction              0.030859
comment_only            0.023144
save_results            0.018858
Name: proportion, dtype: float64

In [10]:
vectorizer = TfidfVectorizer()
clf = lgb.LGBMClassifier()
 
X = scipy.sparse.csr_matrix(train_features[train_columns].values)
X1 = vectorizer.fit_transform(train_features['text'])

X = hstack((X, X1)) 
clf.fit(X, target)

X = scipy.sparse.csr_matrix(validation_features[train_columns].values)
X1 = vectorizer.transform(validation_features['text'])

X = hstack((X, X1)) 
pred = clf.predict(X)
f1_score(pred, validation_features["primary_label"], average='weighted')

0.7194815024105582

In [11]:
X = scipy.sparse.csr_matrix(test_features[train_columns].values)
X1 = vectorizer.transform(test_features['text'])

X = hstack((X, X1)) 

pred = clf.predict(X)
pred

array(['helper_functions', 'modelling', 'data_preprocessing', ...,
       'prediction', 'data_preprocessing', 'prediction'], dtype=object)

In [12]:
# convert array into dataframe
DF = pd.DataFrame(pred)
 
# save the dataframe as a csv file
DF.to_csv("submission.csv")